# 물리 볼륨

- 이전시간에 sdb1,2,3의 마운트 연결 해제 필요.
    - 연결확인 : `lsblk`
    - 마운트 해제 : `umount {Name}` / 예문 : `umount /dev/sdb1`

- `fdisk /dev/sdb1`로 fdisk command 진입
    - command (m for help) : `t`
    - partition number (1-3, default 3): `1`
    - Hex code (type L to list all codes): `L` # 목록 확인
    - Hex code (type L to list all codes) : `8e`\
    changed type of partition 'Linux' to 'Linux LVM'
    - command (m for help) : `p` # 출력하여 확인
    - 2와 3 나머지도 위와 동일한 작업 진행
    - 완료 후
        - command (m for help) : `w`


- 물리 볼륨으로 속성 바꾸기
    - `pvcreate /dev/sdb1`\
    2와 3에도 동일하게 속성 변경
    - `pvdisplay` : 물리볼륨 확인

- 볼륨 그룹 생성하기
    - `vgcreate {test_vg} {/dev/sdb1} {/dev/sdb2}`
    - 볼륨 그룹 확인 : `vgdisplay`

- 논리 볼륨 생성하기
    - `lvcreate -l {pe개수} {논리볼륨이름 - 만들기} {볼륨그룹이름}`
    - 논리 볼륨 확인하기 : `lvdisplay`

### PV 생성 -> VG로 묶어서 하나 처럼 사용 -> LV 사용
> 하나의 물리 드라이브를 여러 개의 논리 드라이브로 분할함으로써 시스템 성능 향상과 용이한 유지보수를 기대할 수 있기 때문

`lvmdiskscan` : 파티셔닝을 한 후에 물리볼륨 생성 -> 변환된 파티션의 수를 확인
> 이부분 재확인 필요

- lvm에 물리볼륨 추가하기
    - 볼륨 그룹에 추가된 디스크 포함시키기
    - `vgextend test_vg /dev/sdb3`
    - lvm 붙여주기
        - `lvextend -l 100%FREE /dev/test_vg/testlv_1`

# 중요) 마운트 하기전에 반드시 `파일시스템`을 만들어야 한다.

- `mkfs -t ext4 /dev/test_vg/testlv_1`
- `mount -t ext4 /dev/test_vg/testlv_1 /mnt`
- `df -h` : 마운트 확인
- `lsblk` : 위치 확인


> Q. resize2fs ?
- 디스크 사이즈 확장 명령어

물리적 공간(디스크) -> 파티셔닝 -> 가상의 공간 (효용성을 위해서 - 반드시 물리적 공간과 연결되어 있어야 한다.)

mapping : 1 대 1 연결\
pv : (파티션 단위로 묶음)\
vg : (pv를 볼륨으로 묶음)\
lvm -> 파일시스템 만들기 -> 마운트

# 머릿속에 남기기 : 파티셔닝 - 파일시스템만들기 - 마운트
> 앞에 배운 작업들은 자주하는 작업이 아니기 때문에 절차를 잊을 수 있다. 하지만 큰 틀을 알아두면 찾아가면서라도 완성이 가능하다.

- LVM 축소는 함부로 하지 말것.
    - 이유는?
        - 데이터가 쪼개져서 드라이브에 들어가는데, 축소되면서 데이터가 어떻게 사라질지 모른다.
        - 정말 축소해야 하는 상황이라면 백업하고 진행하기

### swap memory

- 물리적 디스크 공간을 메모리 공간처럼 사용하는 방식
- 메모리에 발생하는 부하를 줄여주는 방법
- 시스템 RAM + 스왑공간 = 가상메모리
- 원리
    - 프로그램 실행 시 메모리에 로드
    - 메모리 공간 부족 시
    - 오래된 프로세스 스왑공간으로 이동
    - 새 프로세스 로드
- 속도
    - RAM -> SWAP -> 디스크 순서

- 스왑 메모리 만들기
    - 파티션 타입 swap으로 설정
    - `mkswap` 명령어로 스왑 시그니처 포멧

- 스왑 공간 활성화
    - `swapon` 명령어로 활성화
    - `swapoff` 명령어로 비활성화
    - `free` 명령어 사용 스왑 공간 확인

- `/etc/fstab` 으로 영구적 활성화
    - swap은 디렉토리 구조로 접근하지 않기 때문에 마운트포인트를 swap으로 지정한다.
    - 스왑 공간은 백업 및 파일 시스템 점검이 필요없다.

# `Systemd`
> `systemctl`과 연관됨.
- 리눅스 시스템을 컨트롤하기 위한 매니저
- `systemd`는 과거 `init`으로 불렸음.
- `Run level` == `Target`

- Service : 시스템에서 제공해주는 서비스 = 서비스 유닛
- Target : 서비스 유닛을 모아놓은 것 = 서비스 유닛의 집합

> 루트 패스워드 복구할 때, 런레벨 몇인지 확인해보자!

- 시스템 유닛을 나눈 이유는?
    - 세분화되어 있으며 관리하기가 쉽고, 다른 유닛에 영향을 주지 않아서
    - 시스템을 잘 관리하기 위해서

# Log
> 로그 (Log) : 이벤트에 대한 기록

- log파일이 분리된 이유는?
    - 사람이 볼 수 있도록 가공한 것
    - 바이너리 파일

|  |systemd-jounald|rsylogd|
|--|---|--|
|작성된 형태|바이너리|아스키|
|기록 기간|런타임동안 기록|반영구 (기본 4주)|
|읽는 방법|journalctl을 통해 확인가능|`cat` / `head` / `tail` / `vi`|
|저장 위치|/run/log/journal|/var/log|

> 관리자 입장에서는 `파일의 크기`에 대해서 신경써야한다.

`logrotate` <- `crontab`에 예약하는 것이 가능하다.

- `cd /etc`
- `cat rsyslog.conf`

-----

- RULES : 로그의 규칙들을 정의
- Filter : 로그 메세지를 분류하기 위한 기준
- Action : 로그가 저장되는 파일의 위치

### Filter
1. 기능 및 우선순위 기반 필터
    - 로그의 위험도 즉 메시지의 심각수준
    - 0에 가까울수록 시스템에 위험도가 높은 것
2. 속성 기반 필터
    - 호스트 이름, 메세지, 태그 등의 속성을 기준으로 분류
3. 표현 기반 필터
    - 산술 및 문자열 연산 등을 사용한 복잡한 표현식을 사용하기 위해 스크립트를 작성하여 필터링하는 방식

- `journalctl`
- `journalctl {--since} {"yyyy-mm-dd"}`

> nginx 설치하기
- `yum install epel-release`
- `yum install nginx`

> nginx 에러 발생시키고 에러메세지 찾아보기
- nginx 상태 체크 : `systemctl status nginx`
- nginx 실행 : `systemctl start nginx`
- 상태 체크 해보기
- nginx 종료 : `systemctl stop nginx`
- root 계정 로그인 -> /etc/nginx 이동
- `vi nginx.conf` 에서 포트에 `;`추가로 더넣고 오류 만들기
- 이후 nginx 실행
    - 에러 메세지 확인
        - systemctl status nginx.service
        - journal -xe

> ### Tip. 중급, 고급 관리자가 에러 발생시 로그 추적하는 방법
- 로그를 봐도 모르는 문제일 경우
    - 똑같은 에러를 발생 시키는 것을 첫 번째 목표로 삼는다.
- `tail -f /var/log/messages` : 로그 메세지가 쌓이는 창을 띄우고 공백상태로 만든다.
- 에러가 발생한 파일 실행하기
- 해당 파일 찾아가서 `vi` 진입 후 `: set number` 입력하여 문제가 되는 줄 체크하기